In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import r2_score

# Cargar los datos
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Preparar los datos
X = train_data.drop(columns=['Life expectancy ', 'Unnamed: 0', 'Country', 'Year', 'Status'])
y = train_data['Life expectancy ']
X_test = test_data.drop(columns=['Unnamed: 0', 'Country', 'Year', 'Status'])

# Manejar valores faltantes
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)
X_test = imputer.transform(X_test)

# Dividir los datos para validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Corrección del param_grid
param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2']  # Cambié 'auto' por None
}

# RandomizedSearchCV con los parámetros corregidos
random_search_rf = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_distributions=param_grid,
    n_iter=50,
    scoring='r2',
    cv=5,
    random_state=42,
    n_jobs=-1
)

# Entrenamiento del modelo
random_search_rf.fit(X_train, y_train)
best_rf_model = random_search_rf.best_estimator_

# Validación del modelo optimizado
y_val_pred_rf = best_rf_model.predict(X_val)
rf_validation_r2 = r2_score(y_val, y_val_pred_rf)

# Selección de características con RandomForest optimizado
selector_rf = SelectFromModel(best_rf_model, prefit=True, threshold="mean")
X_train_selected = selector_rf.transform(X_train)
X_val_selected = selector_rf.transform(X_val)
X_test_selected = selector_rf.transform(X_test)

# Reentrenar RandomForest con características seleccionadas
rf_model_selected = RandomForestRegressor(n_estimators=200, random_state=42)
rf_model_selected.fit(X_train_selected, y_train)

# Validar el modelo RandomForest con características seleccionadas
y_val_pred_rf_selected = rf_model_selected.predict(X_val_selected)
rf_selected_validation_r2 = r2_score(y_val, y_val_pred_rf_selected)

# Comparar resultados
print("R2 de RandomForest optimizado:", rf_validation_r2)
print("R2 de RandomForest con características seleccionadas:", rf_selected_validation_r2)

# Predicción en el conjunto de prueba
test_predictions = rf_model_selected.predict(X_test_selected)

# Crear el archivo de envío
submission = pd.DataFrame({
    'ID': range(1, len(test_predictions) + 1),
    'Life expectancy': test_predictions
})
submission.to_csv('otris.csv', index=False)

print("Archivo de envío generado: submission.csv")

R2 de RandomForest optimizado: 0.9646425764340475
R2 de RandomForest con características seleccionadas: 0.9476480785665161
Archivo de envío generado: submission.csv
